## Rozwiązywanie układów równań liniowych metodami interacyjnymi 

### Zadanie 1
Zaimplementuj metodę Jacobiego. Podaj warunki jej stosowalności. Wygeneruj co najmniej trzy odpowiednie macierze o różnych wielkościach i sprawdź działanie swojej metody. Zwróć uwagę na zbieżność tej metody. 

### Zadanie 2
Zaimplementuj metodę Gaussa-Seidla i kolejnych nadrelaksacji (successive over-relaxation). Podaj warunki stosowalności tych metod. Przeprowadź badanie działania swoich implementacji analogicznie jak w poprzednim zadaniu. Porównaj zbieżność wszystkich trzech metod. 



In [10]:
import numpy as np

def jacobiego(A,B,numberOfIterations):
    x = np.zeros_like(B)
    
    for it_count in range(numberOfIterations):
        print("Iteration {0}: {1}".format(it_count, x))
        x_new = np.zeros_like(x)
        
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]
        
        #Kończymy działanie jeżeli kolejne iteracje nie zmieniają wyniku
        if np.allclose(x, x_new, atol=1e-10, rtol=0.):
            return x
        x = x_new

A = np.array([[10., -1., 2., 0.],
              [-1., 11., -1., 3.],
              [2., -1., 10., -1.],
              [0.0, 3., -1., 8.]])

B = np.array([6., 25., -11., 15.])
jacobiego(A,B,1000)



Iteration 0: [0. 0. 0. 0.]
Iteration 1: [ 0.6         2.27272727 -1.1         1.875     ]
Iteration 2: [ 1.04727273  1.71590909 -0.80522727  0.88522727]
Iteration 3: [ 0.93263636  2.05330579 -1.04934091  1.13088068]
Iteration 4: [ 1.01519876  1.95369576 -0.96810863  0.97384272]
Iteration 5: [ 0.9889913   2.01141473 -1.0102859   1.02135051]
Iteration 6: [ 1.00319865  1.99224126 -0.99452174  0.99443374]
Iteration 7: [ 0.99812847  2.00230688 -1.00197223  1.00359431]
Iteration 8: [ 1.00062513  1.9986703  -0.99903558  0.99888839]
Iteration 9: [ 0.99967415  2.00044767 -1.00036916  1.00061919]
Iteration 10: [ 1.0001186   1.99976795 -0.99982814  0.99978598]
Iteration 11: [ 0.99994242  2.00008477 -1.00006833  1.0001085 ]
Iteration 12: [ 1.00002214  1.99995896 -0.99996916  0.99995967]
Iteration 13: [ 0.99998973  2.00001582 -1.00001257  1.00001924]
Iteration 14: [ 1.00000409  1.99999268 -0.99999444  0.9999925 ]
Iteration 15: [ 0.99999816  2.00000292 -1.0000023   1.00000344]
Iteration 16: [ 1.0000

In [13]:
import numpy as np
def gaussa_seidla(A,B,numberOfIterations):
    x = np.zeros_like(B)
    
    for it_count in range(numberOfIterations):
        print("Iteration {0}: {1}".format(it_count, x))
        x_new = np.zeros_like(x)
        
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x_new[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]
            
        if np.allclose(x, x_new, rtol=1e-8):
            return x
        x = x_new

#gaussa_seidla(A,B,1000)

def over_relaxation(A, B, omega,numberOfIterations):
    x_new = np.zeros_like(B)
    residual = np.linalg.norm(np.matmul(A, x_new) - B) #Initializacja 
    for k in range(numberOfIterations):
        for i in range(A.shape[0]):
            sigma = 0
            for j in range(A.shape[1]):
                if j != i:
                    sigma += A[i][j] * x_new[j]
            x_new[i] = (1 - omega) * x_new[i] + (omega / A[i][i]) * (B[i] - sigma)
        residual = np.linalg.norm(np.matmul(A, x_new) - B)
        if residual<1e-8:
            return x_new

over_relaxation(A,B,0.5,1000)

array([ 1.,  2., -1.,  1.])